In [1]:
from bs4 import BeautifulSoup
import requests
import re
import os
import urllib
import pandas as pd

import bqplot.pyplot as plt
from bqplot import *

import json
from pandas.io.json import json_normalize

from datetime import datetime

In [2]:
from platform import python_version
print(python_version())

3.5.3


In [3]:
url = 'https://www.politico.com/mapdata-2016/2016-election/primary/results/map/president/'

In [4]:
from fake_useragent import UserAgent
ua = UserAgent()

In [5]:
ua.chrome

'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.3319.102 Safari/537.36'

In [6]:
ua = 'Mozilla/5.0 (X11; CrOS i686 4319.74.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/29.0.1547.57 Safari/537.36'

In [7]:
htmlContent = requests.get(url, headers={'User-Agent': str(ua)})


In [ ]:
soup = BeautifulSoup(htmlContent.text, 'html5lib')
#print(soup.prettify())

In [9]:
# Get id for all states; this gives a list of lists
states = []
states.append([i.get('id') for i in soup.find_all('article',class_="timeline-group")])

In [10]:
# flatten the above result for state ids
states = [y for x in states for y in x]

In [11]:
len(states)

51

In [ ]:
article_main = soup.find(class_="timeline-group",id=states[0])
caucus_primary = str(article_main.findChildren('a')[1].text).lstrip().split()[1]

In [13]:

def scrape_election_results(BeautifulSoup,list):

    global election_dict
    election_dict = {}
    
    for state in range(len(states)):
        state_name = states[state][-2:]
        #print("State:",state)
        article_main = soup.find(class_="timeline-group",id=states[state])
        caucus_primary = str(article_main.findChildren('a')[1].text).lstrip().split()[1]
        election_date = article_main.time.attrs['datetime']

        # get election results for both parties
        div_tags = article_main.find_all('div', class_='results-dataset')
    
        # get names of the parties (from top of the table results)
        party_names = [h.find_all('h5') for h in div_tags]
        flatten_party_names = [y for x in party_names for y in x]

        party = []   
        for p in flatten_party_names:
            party.append(p.text)
            
        # results for BOTH parties - returns a list of two items; first entry if for DEM and second for GOP
        elec_results = article_main.find_all('table',class_="results-table") 

        elec_col_3 = ['Candidate_Name','Pct_Votes_Won','Votes_Won']
        elec_col_names = ['Candidate_Name','Pct_Votes_Won','Votes_Won','Delegates_Won']
        df_ = pd.DataFrame()
        state_result_dict = {}
        
        for res in range(len(elec_results)):
            df_ = pd.read_html(str(elec_results[res]))
            df_json = df_[0].to_json(orient='records')
            #print("\n Printing df_json\n")
            #print(df_json)
            jdata = json.loads(df_json)
            df_ = json_normalize(jdata)
            #print(df_)
            if (len(df_.columns) == 4):
                df_.columns = elec_col_names
            else:
                df_.columns = elec_col_3
                df_['Delegates_Won'] = 'NA'
            df_.fillna(0,inplace=True)
            #df_.reset_index(drop=True, inplace=True)
            df_['Party'] = party[res]
            df_['Election_Type'] = caucus_primary
            df_['Date'] = election_date
            df_['State'] = state_name
            state_result_dict[res] = df_
            #print(state_result_dict[res])
    
        df_state_results = pd.concat(state_result_dict.values(), ignore_index=True)
        
        print("\n")
        election_dict.update({state_name:df_state_results})
        
        
    print(election_dict)      
    return election_dict
    
    



In [16]:
# call the function to scrape all election results data into a dictionary, election_dict which is global
scrape_election_results(soup,states)







































































































{'ND':       Candidate_Name Pct_Votes_Won  Votes_Won  Delegates_Won       Party  \
0  Winner B. Sanders         64.2%        253           14.0  Democratic   
1         H. Clinton         25.6%        101            6.0  Democratic   
2        Uncommitted         10.2%         40            3.0  Democratic   
3    R. De La Fuente          0.0%          0            0.0  Democratic   
4            K. Judd          0.0%          0            0.0  Democratic   
5              Other          0.0%          0            0.0  Democratic   
6          W. Wilson          0.0%          0            0.0  Democratic   

  Election_Type        Date State  
0        Caucus  2016-06-07    ND  
1        Caucus  2016-06-07    ND  
2        Caucus  2016-06-07    ND  
3        Caucus  2016-06-07    ND  
4        Caucus  2016-06-07    ND  
5        Caucus  2016-06-07    ND  
6        Caucus  2016-06-07   

{'AK':       Candidate_Name Pct_Votes_Won  Votes_Won  Delegates_Won       Party  \
 0     Winner T. Cruz         36.4%       7973           12.0  Republican   
 1           D. Trump         33.5%       7346           11.0  Republican   
 2           M. Rubio         15.1%       3318            5.0  Republican   
 3          B. Carson         10.9%       2401            0.0  Republican   
 4          J. Kasich          4.1%        892            0.0  Republican   
 5        Uncommitted          0.0%          0            0.0  Republican   
 6  Winner B. Sanders         81.6%        440           14.0  Democratic   
 7         H. Clinton         18.4%         99            4.0  Democratic   
 8    R. De La Fuente          0.0%          0            0.0  Democratic   
 9        Uncommitted          0.0%          0            2.0  Democratic   
 
   Election_Type        Date State  
 0        Caucus  2016-03-01    AK  
 1        Caucus  2016-03-01    AK  
 2        Caucus  2016-03-01    AK

In [17]:
# reading results from election_dict (which is a dictionary) to a dataframe containing results for all states
df_presidential_results = pd.DataFrame()

all_states_df = [v for k,v in election_dict.items()]
df_presidential_results = pd.concat(all_states_df,ignore_index=True)

In [18]:
#remove string 'Winner' from Candidate Name columns
#df_presidential_results['Candidate_Name'].replace({'Winner':''}, inplace=True, regex=True)

#convert Date to date format
#pd.to_datetime(df_presidential_results['Date'],format='%Y-%m-%d').dt.strftime('%m-%d-%Y')


In [19]:
pres_res_cols = df_presidential_results.columns.tolist()

In [20]:
# reorder the columns and update dataframe
pres_res_cols = ['Date','State','Election_Type','Party','Candidate_Name','Votes_Won','Pct_Votes_Won','Delegates_Won']
df_presidential_results = df_presidential_results[pres_res_cols]

In [21]:
# setting multiple index
df_pr_mi = df_presidential_results.set_index(['Date','Election_Type','State','Party'])

In [56]:
# indexed by Date, Election_Type, State, Party
df_pr_mi.head(5)

Candidate_Name  Votes_Won  \
Date       Election_Type State Party                                      
2016-06-07 Caucus        ND    Democratic  Winner B. Sanders        253   
                               Democratic         H. Clinton        101   
                               Democratic        Uncommitted         40   
                               Democratic    R. De La Fuente          0   
                               Democratic            K. Judd          0   

                                          Pct_Votes_Won Delegates_Won  
Date       Election_Type State Party                                   
2016-06-07 Caucus        ND    Democratic         64.2%            14  
                               Democratic         25.6%             6  
                               Democratic         10.2%             3  
                               Democratic          0.0%             0  
                               Democratic          0.0%             0

In [23]:
# BEFORE removing the substring 'Winner' from Candidate Name.

df_state_winner = pd.DataFrame()
df_state_winner = df_presidential_results[df_presidential_results['Candidate_Name'].str.contains('Winner')]
df_state_winner['Candidate_Name'].replace({'Winner':''}, inplace=True, regex=True)

# this df lists winners for each state from each party; Some states have results only for one party
# This df is used to assign colors to states in the US map.

df_state_winner = df_state_winner[df_state_winner.Election_Type != 'Caucus']
df_state_winner = df_state_winner[df_state_winner.Election_Type != 'Caucuses']

C:\Anaconda3\envs\ds19\lib\site-packages\pandas\core\generic.py:4619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [24]:
# replace all candidate names with winner for that party
# this df is used to indicate colors for state won by a particular party

df_state_winner.loc[df_state_winner.Party == 'Democratic','Candidate_Name'] = 'Clinton'
df_state_winner.loc[df_state_winner.Party == 'Republican','Candidate_Name'] = 'Trump'

In [25]:
# dropping columns not relevant to display results on the map
df_state_winner.drop(columns=['Date','Delegates_Won','Election_Type'],inplace=True)


In [26]:
# copy the pivoted results to a new df
df_winner = pd.DataFrame()
df_winner = df_state_winner.pivot(index='State',columns='Candidate_Name')

In [27]:
df_winner.columns = list(map("_".join, df_winner.columns))

In [28]:
df_winner['State'] = df_winner.index

# execute this only ONCE
df_winner.reset_index(drop=True,inplace=True)

In [29]:
df_winner.drop(columns=['Party_Clinton','Party_Trump'],inplace=True)

In [30]:
# rename columns
df_winner.columns = ['Clinton','Trump','% Clinton', '% Trump','State']

In [31]:
# this df does not have all the states, since we removed results for Caucus states
len(df_winner)
df_winner.head(5)

,Clinton,Trump,% Clinton,% Trump,State
0,309928,371735,77.8%,43.4%,AL
1,144580,133144,66.3%,32.8%,AR
2,235697,249916,57.6%,47.1%,AZ
3,1940580,1174829,55.8%,75.3%,CA
4,170075,123367,51.8%,57.9%,CT


In [32]:
# retain only the Primaries results and drop other rows
df_presidential_results.drop(df_presidential_results[(df_presidential_results.Election_Type == 'Caucus') | (df_presidential_results.Election_Type == 'Caucuses')].index,inplace=True)


In [33]:
#remove the substring winner from candidate names
df_presidential_results['Candidate_Name'].replace({'Winner':''}, inplace=True, regex=True)

In [34]:
# saving the df with all results
#df_presidential_results.to_csv('pr_data_files/df_all_results.csv',index=None,header=True)

In [35]:
df_presidential_results.head(5)

,Date,State,Election_Type,Party,Candidate_Name,Votes_Won,Pct_Votes_Won,Delegates_Won
7,2016-03-01,AL,Primaries,Democratic,H. Clinton,309928,77.8%,51
8,2016-03-01,AL,Primaries,Democratic,B. Sanders,76399,19.2%,9
9,2016-03-01,AL,Primaries,Democratic,Uncommitted,9534,2.4%,0
10,2016-03-01,AL,Primaries,Democratic,M. O'Malley,1489,0.4%,0
11,2016-03-01,AL,Primaries,Democratic,R. De La Fuente,814,0.2%,0


In [36]:
# from the saved pres_state_results.csv file, create df for democrat and republican each

In [37]:
## reading in from a file and not from the existing df_presidential_results, as this csv file has state ID and state Names
all_res = pd.read_csv('pr_data_files/pres_state_results.csv')
df_all_res = pd.DataFrame(all_res)

In [57]:
df_DEM = pd.DataFrame()
df_GOP = pd.DataFrame()

In [58]:
# save results for democratic and republican parties in different dataframes
df_DEM = df_all_res[df_all_res['Party'] == 'Democratic']
df_GOP = df_all_res[df_all_res['Party'] == 'Republican']

In [40]:
 ## Now, Plot the Map ##

In [59]:
from bqplot import Tooltip, AlbersUSA, OrdinalColorScale, ColorAxis
from bqplot.market_map import MarketMap
import bqplot
import ipywidgets as ipw
from IPython.display import display, clear_output
from ipywidgets import HBox, VBox

In [60]:
# creating map for DEM only.
# reading data from df_DEM

global df_DEM
global df_GOP

In [61]:
df_DEM.head(5)

,Date,State,Election_Type,Party,Candidate_Name,Votes_Won,Pct_Votes_Won,Delegates_Won,Name,id
0,6/7/2016,CA,Primaries,Democratic,H. Clinton,1940580,55.80%,325.0,California,6
1,6/7/2016,CA,Primaries,Democratic,B. Sanders,1502043,43.20%,222.0,California,6
2,6/7/2016,CA,Primaries,Democratic,W. Wilson,8709,0.30%,0.0,California,6
3,6/7/2016,CA,Primaries,Democratic,M. Steinberg,7905,0.20%,0.0,California,6
4,6/7/2016,CA,Primaries,Democratic,R. De La Fuente,5780,0.20%,0.0,California,6


In [62]:
df_DEM.drop(columns=['Date','Election_Type','Party'],inplace=True)

C:\Anaconda3\envs\ds19\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [63]:
df_temp = df_DEM[df_DEM['Candidate_Name'] == ' H. Clinton']
df_temp_1 = df_DEM[df_DEM['Candidate_Name'] == 'H. Clinton']
df_temp_2 = df_DEM[df_DEM['Candidate_Name'] == 'B. Sanders']
df_temp_3 = df_DEM[df_DEM['Candidate_Name'] == ' B. Sanders']
all_DEM = [df_temp,df_temp_1,df_temp_2,df_temp_3]

In [64]:
df_DEM_win = pd.concat(all_DEM,ignore_index=True)

In [47]:
#df_DEM_win.to_csv('pr_data_files/df_DEM_win.csv',header=True,index=False)

In [65]:
DEM_res = pd.read_csv('pr_data_files/DEM_win.csv')

In [145]:
###############################################################################
# create an array for DEM wins - Sanders vs Clinton
winner_DEM = np.array(['Clinton'] * DEM_res.shape[0])
winner_DEM[(DEM_res['Sanders_Votes'] > DEM_res['Clinton_Votes']).values] = 'Sanders'

##############################################

In [143]:
df_DEM_res = pd.DataFrame(DEM_res)
global df_DEM_res

In [141]:
def DEM_get_state_data(code):
    # use the dataframe
    curr_res = pd.DataFrame()
    curr_res = df_DEM_res[df_DEM_res['id'] == code]
       
    return curr_res

In [146]:
out_dem = ipw.Output()
def DEM_hover_callback(name,value):  
    df_disp = pd.DataFrame()
    df_disp = DEM_get_state_data(value['data']['id'])
    out_dem.clear_output()
    with out_dem:
        display(ipw.HTML(df_disp.to_html()))
        

    

In [ ]:

geo = AlbersUSA()
DEM_st_color = OrdinalColorScale(domain=['Sanders', 'Clinton'], colors=['green', 'blue'])
DEM_st_winner = dict(zip(DEM_res['id'], list(winner_DEM)))

DEM_map = Map(map_data=topo_load('map_data/USStatesMap.json'), scales={'projection': geo, 'color':DEM_st_color},
                 color=DEM_st_winner, tooltip=out_dem)

DEM_map.interactions = {'click': 'select', 'hover': 'tooltip'}
DEM_map.on_hover(DEM_hover_callback)

Figure(marks=[DEM_map], title='US Elections 2016 - DEM')


#### Not sure why I can't get the color scheme for DEM ! ######

In [158]:
df_gop_1 = df_GOP[df_GOP['Candidate_Name'] == ' D. Trump']
df_gop_2 = df_GOP[df_GOP['Candidate_Name'] == 'D. Trump']
df_gop_3 = df_GOP[df_GOP['Candidate_Name'] == ' T. Cruz']
df_gop_4 = df_GOP[df_GOP['Candidate_Name'] == 'T. Cruz']
all_GOP = [df_gop_1,df_gop_2,df_gop_3,df_gop_4]

In [159]:
df_GOP_win = pd.concat(all_GOP,ignore_index=True)

In [160]:
df_GOP_win.drop(columns=['Election_Type','Party','Date'],inplace=True)

In [72]:
#df_GOP_win.to_csv('pr_data_files/df_GOP_win.csv',header=True,index=False)


In [161]:
GOP_res = pd.read_csv('pr_data_files/GOP_win.csv')

In [166]:
GOP_res.head(5)

,State,id,Trump_Votes,Cruz_Votes,Trump_%,Cruz_%
0,AL,2,"371,735","180,608",43.4%,21.1%
1,AR,2,"133,144","123,873",32.8%,30.5%
2,AZ,4,"249,916","132,147",47.1%,24.9%
3,CA,6,"1,174,829","144,125",75.3%,9.2%
4,CT,9,"123,367","24,969",57.9%,11.7%


In [167]:
## create an array for GOP wins - Trump vs. Cruz
winner_GOP = np.array(['Trump'] * GOP_res.shape[0])
winner_GOP[(GOP_res['Cruz_Votes'] > GOP_res['Trump_Votes']).values] = 'Cruz'

##############################################

In [162]:
df_GOP_res = pd.DataFrame(GOP_res)
global df_GOP_res

In [163]:
def GOP_get_state_data(code):
    # use the dataframe
    curr_res = pd.DataFrame()
    curr_res = df_GOP_res[df_GOP_res['id'] == code]
       
    return curr_res

In [164]:
out_gop = ipw.Output()
def GOP_hover_callback(name,value):  
    df_disp = pd.DataFrame()
    df_disp = GOP_get_state_data(value['data']['id'])
    out_gop.clear_output()
    with out_gop:
        display(ipw.HTML(df_disp.to_html()))
        

In [165]:
#### GOP map with a widget ##########
     
geo = AlbersUSA()
GOP_st_color =  OrdinalColorScale(domain=['Trump', 'Cruz'], colors=['red', 'yellow'])

GOP_st_winner = dict(zip(GOP_res['id'], list(winner_GOP)))

GOP_map_1 = Map(map_data=topo_load('map_data/USStatesMap.json'), scales={'projection': scale_geo, 'color':scale_states_GOP},
                 color=states_winner_GOP, tooltip=out_gop)

GOP_map_1.interactions = {'click': 'select', 'hover': 'tooltip'}

GOP_map_1.on_hover(GOP_hover_callback)
Figure(marks=[GOP_map_1], title='US Elections 2016 - GOP!!')

Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, marks=[Map(color={2: 'Trump', 4: 'Trump'…